In [1]:
import pandas as pd

In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

In [3]:
# Carregar o arquivo Excel
file_path = "Template - Requisito de Software v02.xlsx"
xls = pd.ExcelFile(file_path)

# Carregar abas específicas
sobre_documento = pd.read_excel(xls, sheet_name="Sobre o documento")
glossario = pd.read_excel(xls, sheet_name="1-Glossário")
requisitos = pd.read_excel(xls, sheet_name='2-Requisitos de Software', skiprows= len(xls.sheet_names)+1)
lists = pd.read_excel(xls, sheet_name="3-Lists")
versoes = pd.read_excel(xls, sheet_name="4-Versões")
guia = pd.read_excel(xls, sheet_name="5-Guia")

#requisitos= requisitos.dropna(how='all')
# Exibir as primeiras linhas da aba de requisitos
print(requisitos.head())

requisitos_texto = requisitos.to_string(index=False)

        Req ID             Subsistema Funcionalidade  \
0  SYSRS-00001          Inicialização  Inicialização   
1  SYSRS-00002          Inicialização  Inicialização   
2  SYSRS-00003          Inicialização     Auto teste   
3  SYSRS-00004  Configuração da probe   Configuração   
4  SYSRS-00005  Configuração da probe   Configuração   

                                           Requisito  \
0  O sistema deve ser ligado após a conexão segur...   
1  O sistema deve inicializar automaticamente com...   
2  O sistema deve realizar um auto teste ao ligar...   
3  O sistema deve permitir a configuração de dife...   
4  O sistema deve reconhecer automaticamente quan...   

                                         Comentários Status  Histórico  \
0                                                NaN   Novo        NaN   
1  Presets devem incluir ajustes automáticos de g...   Novo        NaN   
2  O auto teste deve incluir verificações de cone...   Novo        NaN   
3                             

/opt/conda/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [25]:
from langchain.docstore.document import Document

# Criar lista de documentos
docs = []
for _, row in requisitos.iterrows():
    content = f"ID: {row['Req ID']}\nSubsistema: {row['Subsistema']}\nFuncionalidade: {row['Funcionalidade']}\nRequisito: {row['Requisito']}"
    docs.append(Document(page_content=content, metadata={"ID": row["Req ID"]}))

In [26]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

# Criar embeddings e base vetorial
embeddings = OllamaEmbeddings(model="llama3.3:latest")
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("requisitos_index")

In [ ]:
response

In [31]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

# Recarregar a base de dados
vectorstore = FAISS.load_local("requisitos_index", embeddings, allow_dangerous_deserialization=True)

# Criar ferramenta de busca nos requisitos
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=Ollama(model="llama3.3:latest"), retriever=retriever)

search_tool = Tool(name="Buscar Requisitos", func=qa_chain.run, description="Busca e analisa requisitos do sistema.")

# Criar o agente
llm = Ollama(model="llama3.3:latest")
agent = initialize_agent(tools=[search_tool], llm=llm, agent="zero-shot-react-description", verbose=True)

# Fazer perguntas ao agente
response = agent.run("Quais requisitos falam sobre inicialização?")
print(response)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Para encontrar os requisitos que falam sobre inicialização, devo buscar dentro do sistema pelos termos relacionados à inicialização.

Action: Buscar Requisitos
Action Input: tags=["inicialização"], metadata={"contexto": "sistema"}
Observation: Com base nos requisitos fornecidos, os IDs relacionados à inicialização são SYSRS-00001 e SYSRS-00003.

- O sistema deve ser ligado após a conexão segura da sonda (probe) ao dispositivo principal (SYSRS-00001).
- O sistema deve realizar um auto teste ao ligar, verificando o funcionamento dos módulos FPGA, processador e bateria (SYSRS-00003).

Esses são os requisitos relacionados à inicialização do sistema.
Thought:Com base na observação, identifiquei dois requisitos específicos que abordam a inicialização do sistema. Agora posso fornecer uma resposta detalhada sobre quais são esses requisitos.

Action: Buscar Requisitos
Action Input: ids=["SYSRS-00001", "SYSRS-00003"], metadata={"contexto": "detalhes"}
Observation: Não tenho informações suficient

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Os dois requisitos específicos que falam sobre a inicialização do sistema são:

1. **SYSRS-00001**: O sistema deve ser ligado após a conexão segura da sonda (probe) ao dispositivo principal.
2. **SYSRS-00003**: O sistema deve realizar um auto teste ao ligar, verificando o funcionamento dos módulos FPGA, processador e bateria.

Esses requisitos destacam a importância da conexão segura da sonda para iniciar o sistema e garantir que todos os componentes críticos estejam funcionando corretamente antes de começar a operar.`

In [33]:
# Função para validar requisitos
def validar_requisitos(texto):
    """
    Analisa o texto de requisitos e verifica os seguintes critérios:
    - Clareza e Precisão: Detecta termos subjetivos ou ambíguos.
    - Sentenças Curtas: Verifica se frases são muito longas.
    - Requisitos Individuais: Identifica frases contendo múltiplos requisitos.
    - Independência: Detecta dependências usando palavras como 'e', 'mas', 'ou'.
    - Consistência: Verifica se a terminologia está uniforme.
    - Completude: Checa se há critérios de aceitação.
    """
    problemas = []

    # Detectar subjetividade e ambiguidade
    termos_subjetivos = ["apropriado", "rápido", "eficiente", "melhor", "intuitivo", "fácil"]
    for termo in termos_subjetivos:
        if termo in texto.lower():
            problemas.append(f"O requisito contém termo subjetivo: '{termo}'.")

    # Verificar comprimento das frases
    frases = texto.split(". ")
    for frase in frases:
        if len(frase) > 150:
            problemas.append(f"A seguinte frase é muito longa ({len(frase)} caracteres): '{frase}'.")

    # Identificar múltiplos requisitos
    conectivos = [" e ", " além disso ", " bem como ", " também "]
    for palavra in conectivos:
        if palavra in texto:
            problemas.append(f"O requisito pode conter múltiplas exigências devido ao uso de '{palavra.strip()}'.")

    # Verificar dependências entre requisitos
    referencias = [" mas ", " porém ", " entretanto ", " contudo ", " deve ser compatível com ", " requer "]
    for ref in referencias:
        if ref in texto:
            problemas.append(f"O requisito pode depender de outro devido ao uso de '{ref.strip()}'.")

    # Checar critério de aceitação (mínimo: deve conter 'deve' ou 'requer')
    if not any(kw in texto.lower() for kw in ["deve", "requer", "será considerado completo quando"]):
        problemas.append("O requisito pode estar incompleto, pois não define critérios de aceitação.")

    if problemas:
        return "⚠️ Problemas identificados:\n" + "\n".join(problemas)
    return "✅ O requisito parece estar bem definido."

In [36]:
# Criar ferramenta de validação de requisitos
validation_tool = Tool(
    name="Validar Requisitos",
    func=validar_requisitos,
    description="Avalia a qualidade de novos requisitos verificando clareza, completude e consistência. Forneça um requisito para análise."
)


search_tool = Tool(name="Buscar Requisitos", func=qa_chain.run, description="Busca e analisa requisitos do sistema.")

In [37]:
# Criar o agente com ambas as ferramentas
llm = Ollama(model="llama3.3:latest")
agent = initialize_agent(
    tools=[search_tool, validation_tool],  # Mantemos as duas ferramentas
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [38]:
print("\n✅ Validando um requisito específico...")
requisito_exemplo = "O sistema deve ser intuitivo e rápido, garantindo a melhor experiência do usuário."
response_validacao = agent.run(f"Valide este requisito: {requisito_exemplo}")
print(response_validacao)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



✅ Validando um requisito específico...
Para validar o requisito "O sistema deve ser intuitivo e rápido, garantindo a melhor experiência do usuário", é necessário analisar se ele atende aos critérios de clareza, completude e consistência.

Action: Validar Requisitos
Action Input: O sistema deve ser intuitivo e rápido, garantindo a melhor experiência do usuário.
Observation: ⚠️ Problemas identificados:
O requisito contém termo subjetivo: 'rápido'.
O requisito contém termo subjetivo: 'melhor'.
O requisito contém termo subjetivo: 'intuitivo'.
O requisito pode conter múltiplas exigências devido ao uso de 'e'.
Thought:Com base na observação do resultado da validação, é possível identificar que o requisito apresenta problemas relacionados a termos subjetivos ("rápido", "melhor" e "intuitivo") e pode conter múltiplas exigências devido ao uso do conectivo "e". Isso sugere que o requisito não está claro ou completo o suficiente, pois diferentes pessoas podem interpretar esses termos de maneiras

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Parece que há um loop infinito de pensamentos sobre a validação do requisito "O sistema deve ser intuitivo e rápido, garantindo a melhor experiência do usuário". Vamos tentar quebrar esse ciclo e oferecer uma análise mais detalhada e conclusiva sobre o assunto.

### Análise do Requisito

O requisito mencionado busca combinar três aspectos importantes para o desenvolvimento de sistemas: intuitividade, rapidez e experiência do usuário. Cada um desses aspectos é crucial para o sucesso de um sistema, especialmente em ambientes competitivos onde a usabilidade e a satisfação do usuário são fatores determinantes.

1. **Intuitividade**: Refere-se à capacidade do sistema de ser facilmente compreendido e utilizado por seus usuários sem a necessidade de treinamento extensivo. A int

In [4]:
print(requisitos[:2])  # Mostrar os dois primeiros requisitos

        Req ID     Subsistema Funcionalidade  \
0  SYSRS-00001  Inicialização  Inicialização   
1  SYSRS-00002  Inicialização  Inicialização   

                                           Requisito  \
0  O sistema deve ser ligado após a conexão segur...   
1  O sistema deve inicializar automaticamente com...   

                                         Comentários Status  Histórico  \
0                                                NaN   Novo        NaN   
1  Presets devem incluir ajustes automáticos de g...   Novo        NaN   

  Versão de modificação     Origem  Jira issue  Test Case Jira  
0                   v_1        NaN         NaN             NaN  
1                   v_1  Documento         NaN             NaN  


In [5]:
regras = '''

### Regras de Escrita de Requisitos

    #### 1.1 Clareza e Precisão
    - Os requisitos devem ser claros, precisos e não genéricos.
      - Evite palavras subjetivas (ex.: "rápido", "seguro").
      - Seja explícito: "Explícito é melhor que implícito".
      - Exemplo Ruim: "O sistema deve aceitar senhas curtas."
      - Exemplo Bom: "O sistema deve aceitar senhas com até 15 caracteres."

    - Todos os requisitos devem ter um critério de aceitação claro.
      - Exemplo: "O sistema deve permitir login com e-mail e senha. Critério: O usuário deve ser redirecionado para a página inicial após o login."

    #### 1.2 Sentenças Curtas
    - Use declarações curtas e objetivas.
      - Exemplo Bom: "O sistema deve registrar a temperatura do sensor a cada 2 minutos."

    #### 1.3 Individualidade
    - Cada requisito deve conter apenas um elemento ou funcionalidade.
      - Evite usar palavras como "e", "ou", "mas".
      - Exemplo Ruim: "O sistema deve permitir reservar voos e comprar passagens."
      - Exemplo Bom: Divida em dois requisitos:
        - "O sistema deve permitir reservar voos."
        - "O sistema deve permitir comprar passagens."

    #### 1.4 Independência
    - Um requisito não deve depender de outro.
      - Exemplo Ruim: "A lista de voos deve ser classificada por preço, do menor para o maior." (Depende de outro requisito sobre a lista de voos.)
      - Exemplo Bom: "A lista de voos disponíveis deve incluir números de voo e ser classificada por preço, do menor para o maior."

    #### 1.5 Consistência
    - Use terminologia consistente.
      - Exemplo Ruim: Usar "salvar", "registrar" e "gravar" para descrever a mesma ação.
      - Exemplo Bom: Escolha um termo e use-o sempre.

    #### 1.6 Completude
    - Inclua todas as informações necessárias para implementação e verificação.
      - Exemplo Bom: Inclua tabelas ou imagens quando necessário.
      - Exemplo: Um requisito pode incluir uma tabela com condições de ativação, prioridade, mensagem, etc.

    #### 1.7 Verificabilidade
    - Os requisitos devem ser testáveis (passou/falhou).
      - Evite adjetivos vagos (ex.: "robusto", "seguro") ou advérbios imprecisos (ex.: "rapidamente").
      - Exemplo Ruim: "O sistema deve resistir ao uso simultâneo por muitos usuários."
      - Exemplo Bom: "O sistema deve suportar até 100 usuários simultâneos."

    #### 1.8 Não Repetição
    - Evite duplicar requisitos.
      - Se necessário, referencie outro requisito já declarado.
      - Exemplo Ruim: "Um calendário deve estar disponível para ajudar na escolha da data do voo." E "O sistema deve exibir um calendário pop-up ao inserir qualquer data."
      - Exemplo Bom: Combine os dois requisitos em um único.

### Regras de Organização e Outras Dicas

    #### 2.1 Hierarquia
    - Escreva os requisitos de forma hierárquica/estruturada.
      - Siga uma estrutura lógica (ex.: V-model).

    #### 2.2 Neutralidade em Termos de Implementação
    - Declare *o que* o sistema deve fazer, não *como* fazê-lo.
      - Exemplo Ruim: "O sistema deve usar Python para processamento."
      - Exemplo Bom: "O sistema deve processar dados em tempo real."

    #### 2.3 Armadilha da Implementação
    - Evite detalhar como algo será implementado nos requisitos de alto nível.
      - Exemplo Ruim: "O sistema deve usar MQTT para comunicação."
      - Exemplo Bom: "O sistema deve permitir comunicação em tempo real."

    #### 2.4 Rastreabilidade
    - Cada requisito deve ter um ID único.
      - Associações entre requisitos pai e filhos devem ser registradas.
      - Exemplo: "REQ001 → REQ002, REQ003".

    #### 2.5 Agrupamento por Assunto
    - Agrupe requisitos por funcionalidade ou assunto.
      - Exemplo: Agrupar requisitos relacionados a alarmes ou funcionalidades específicas.

    #### 2.6 Controle de Alterações
    - Utilize recursos de controle de alterações.
      - Ative o controle de alterações no Word/Excel para acompanhar mudanças.

    #### 2.7 Check-Out
    - Utilize o recurso de check-out.
      - Evite alterações paralelas no documento.

    #### 2.8 Comunicação
    - Comunique todos os envolvidos a cada nova versão.
      - Informe sobre atualizações para garantir alinhamento.'''


In [12]:
#llm = OllamaLLM(model="llama3.3", temperature = 0.3, max_new_tokens = 2023, max_length=2023,)
llm = OllamaLLM(model="llama3.3", temperature = 0.3)
#llm = OllamaLLM(model="deepseek-r1:70b", temperature = 0.1)

In [15]:
from langchain.chains import LLMChain
# Criar um template de prompt para análise de requisitos
template = """Você é um assistente especializado em análise de requisitos de software. 

Reestruture essa tabela de requisitos:
{requisitos_texto}

"""

prompt = PromptTemplate(input_variables=["requisitos_texto"], template=template)

# Criar a cadeia de análise
chain = LLMChain(llm=llm, prompt=prompt)

# Rodar a análise para os requisitos carregados
response = chain.run(requisitos_texto=requisitos_texto) 

print(response)  # Exibir a resposta do modelo
#print(response[response.find('</think>')+8:])

<think>
Okay, so I'm looking at this list of requirements labeled from SYSRS-00001 to SYSRS-00065. Each one seems to be a system requirement, but beyond the labels, there's not much detail provided. My task is to analyze these and figure out what they mean or how they can be used.

First off, I notice that each requirement starts with "SYSRS-" followed by a five-digit number. That probably stands for System Requirement Specification. So, these are all part of some system's requirements document. But without any descriptions or context, it's hard to tell exactly what each one is about.

I think the first step is to try and categorize these requirements. Maybe they're functional or non-functional. Functional requirements would be about what the system should do, like user authentication or data processing. Non-functional might cover performance, security, etc. But again, without more info, it's just a guess.

Looking at the numbering, from 00001 to 00065, that's 65 requirements. That see

In [13]:
from langchain.chains import LLMChain
# Criar um template de prompt para análise de requisitos
template = """Você é um assistente especializado em análise de requisitos de software. Sua tarefa é avaliar a tabela de requisitos com base nas regras fornecidas abaixo:

Regras: {regras} 

Tabela de requisitos a serem avaliados:
{requisitos_texto}

Os campos com "NaN" (Not a Number) são relativos a requisitos que ainda não estão completos ou estão sendo editados ainda, foque apenas nas linhas que tem as colunas REQ ID e Requisito completas com dados. 
Por favor, faça uma análise de cada requisito identificado pelo Req Id e presente na coluna Requisito e responda às seguintes perguntas:
1. Há requisitos que violam as regras fornecidas? Liste-os e explique por quê.
2. Há redundâncias ou conflitos entre os requisitos? Explique.
3. Há lacunas ou requisitos incompletos? Sugira melhorias.
4. Os requisitos estão organizados de forma clara e consistente? Comente.
"""

prompt = PromptTemplate(input_variables=["regras","requisitos_texto"], template=template)

# Criar a cadeia de análise
chain = LLMChain(llm=llm, prompt=prompt)

# Rodar a análise para os requisitos carregados
response = chain.run(regras=regras, requisitos_texto=requisitos_texto) 

print(response)  # Exibir a resposta do modelo
#print(response[response.find('</think>')+8:])



### Análise dos Requisitos

#### 1. **Requisitos que violam as regras fornecidas?**
   - **REQ-001**: "O sistema deve permitir que os usuários façam login usando seu e-mail e senha."
     - **Violação Potencial**: Não há menção a segurança adicional, como autenticação de dois fatores (MFA) ou requisitos de complexidade de senha. Isso pode violar as melhores práticas de segurança.
   - **REQ-007**: "O sistema deve permitir o cadastro de usuários."
     - **Violação Potencial**: Não há especificação sobre a proteção de dados pessoais, o que pode violar regulamentações como o GDPR ou LGPD.

#### 2. **Redundâncias ou conflitos entre os requisitos?**
   - **REQ-003** e **REQ-009**: Ambos lidam com informações de perfil do usuário, podendo ser redundantes se não houver clareza sobre suas diferenças.
   - **REQ-002** e **REQ-005**: A dashboard e a funcionalidade de busca podem ter sobreposição se a busca for uma parte integrante da dashboard.

#### 3. **Lacunas ou requisitos incompletos?**


In [14]:
print(response[: response.find('</think>')])

<think>
Okay, so I'm trying to help analyze these requirements provided in the query. The user has given a table with several REQ IDs and their corresponding Requisito (Requirement) descriptions. However, many of the entries have "NaN," which indicates they're incomplete or still being worked on. My task is to focus only on the rows where both REQ ID and Requisito are filled.

First, I need to go through each completed requirement and check for any violations of given rules. Then, look for redundancies or conflicts between them. Next, identify if there are any gaps or incomplete parts that need improvement. Finally, assess if the requirements are organized clearly and consistently.

Starting with the first few REQ IDs:

1. **REQ-001**: The system must allow users to log in using their email and password.
   - This seems straightforward, but I should check if it violates any rules. Maybe security protocols require multi-factor authentication or specific password policies, which aren't m